In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
from keras_applications.resnet import ResNet34
from tensorflow.keras import layers, models

# Load ResNet34 without the top classification layer
base_model = ResNet34(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the convolutional base
base_model.trainable = False


ImportError: cannot import name 'ResNet34' from 'keras_applications.resnet' (C:\Users\redfr\elevenvenv\Lib\site-packages\keras_applications\resnet.py)

In [ ]:
# Add custom layers
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),  # Replace Flatten for global pooling
    layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # Sigmoid activation for binary classification
])

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',  # Binary classification loss
    metrics=['accuracy']
)

In [ ]:
# Directory paths for training and validation data
train_dir = 'C:\\Users\\redfr\\Downloads\\archive\\for-2sec\\for-2seconds\\training-images'
val_dir = 'C:\\Users\\redfr\\Downloads\\archive\\for-2sec\\for-2seconds\\testing-images'

# Data augmentation for the training set
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,  # Normalize pixel values
    rotation_range=15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

# Only rescale for validation data
val_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Load training and validation data
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'  # Binary classification
)

val_data = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

In [ ]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10  # Adjust epochs as needed
)

In [ ]:
base_model.trainable = True

# Recompile the model with a lower learning rate for fine-tuning
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Fine-tune the model
history_fine_tune = model.fit(
    train_data,
    validation_data=val_data,
    epochs=5  # Additional fine-tuning epochs
)

In [ ]:
test_dir ='C:\\Users\\redfr\\Downloads\\archive\\for-2sec\\for-2seconds\\testing-images'

test_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_loss, test_accuracy = model.evaluate(test_data)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")